In [1]:
%%capture
pip install numpy pandas

In [2]:
import numpy as np
import pandas as pd

#### **1. Do not consider people who are in cohort_users after completing form_entry**

In [3]:
form_entry = pd.read_csv('../sources/form_entry.csv', low_memory=False)

In [4]:
form_entry.columns

Index(['id', 'first_name', 'last_name', 'email', 'phone', 'course',
       'client_comments', 'language', 'utm_url', 'utm_medium', 'utm_campaign',
       'street_address', 'country', 'city', 'latitude', 'longitude', 'state',
       'zip_code', 'storage_status', 'created_at', 'updated_at', 'contact_id',
       'lead_type', 'location', 'referral_key', 'automations', 'tags', 'gclid',
       'browser_lang', 'fb_ad_id', 'fb_adgroup_id', 'fb_form_id',
       'fb_leadgen_id', 'fb_page_id', 'utm_source', 'academy_id',
       'deal_status', 'sentiment', 'ac_contact_id', 'ac_deal_id', 'user_id',
       'won_at', 'ac_expected_cohort', 'current_download',
       'lead_generation_app_id', 'utm_content', 'storage_status_text',
       'custom_fields', 'sex', 'utm_placement', 'utm_plan', 'utm_term',
       'ac_deal_owner_full_name', 'ac_deal_owner_id',
       'ac_expected_cohort_date', 'ac_deal_amount', 'ac_deal_currency_code'],
      dtype='object')

In [5]:
cohort_users = pd.read_csv('../sources/cohort_users.csv')

In [6]:
cohort_users

,id,role,finantial_status,educational_status,created_at,updated_at,cohort_id,user_id,watching
0,1,STUDENT,LATE,SUSPENDED,2020-08-19 23:06:31.519727+00:00,2020-08-19 23:06:31.519746+00:00,9,14,False
1,3,STUDENT,LATE,SUSPENDED,2020-08-19 23:06:31.532619+00:00,2020-08-19 23:06:31.532644+00:00,31,14,False
2,4,STUDENT,LATE,SUSPENDED,2020-08-19 23:06:31.538438+00:00,2020-08-19 23:06:31.538457+00:00,4,14,False
3,5,STUDENT,NaN,GRADUATED,2020-08-19 23:06:31.550397+00:00,2020-08-19 23:06:31.550416+00:00,9,15,False
4,7,STUDENT,NaN,GRADUATED,2020-08-19 23:06:31.565350+00:00,2020-08-19 23:06:31.565371+00:00,9,16,False
...,...,...,...,...,...,...,...,...,...
7729,18592,STUDENT,NaN,DROPPED,2023-01-31 18:21:04.258037+00:00,2023-01-31 18:21:18.631759+00:00,397,5545,False
7730,18594,STUDENT,NaN,NaN,2023-01-31 19:14:17.126300+00:00,2023-01-31 19:14:17.126314+00:00,61,6004,False
7731,18593,STUDENT,NaN,ACTIVE,2023-01-31 19:08:18.651832+00:00,2023-01-31 19:15:43.203963+00:00,523,3617,False
7732,18595,STUDENT,NaN,NaN,2023-01-31 21:23:50.539551+00:00,2023-01-31 21:23:50.539572+00:00,61,6005,False


In [7]:
auth = pd.read_csv('../sources/auth_user.csv')

In [8]:
cohort_users_auth = cohort_users.merge(auth, left_on='user_id', right_on='id', how='left')

In [9]:
cohort_users_auth.columns

Index(['id_x', 'role', 'finantial_status', 'educational_status', 'created_at',
       'updated_at', 'cohort_id', 'user_id', 'watching', 'id_y', 'first_name',
       'last_name', 'email', 'date_joined'],
      dtype='object')

In [10]:
cohort_users_auth = cohort_users_auth[['email', 'role', 'created_at', 'cohort_id']]

In [11]:
cohort_users_auth = cohort_users_auth[cohort_users_auth['role'].str.lower()=='student']

In [12]:
# Delete "+00:00"
def correct_date(time):
    if pd.isna(time):
         return time
    else:
        sep = '+'
        stripped = time.split(sep, 1)[0]
        return stripped

In [13]:
cohort_users_auth['created_at'] = cohort_users_auth['created_at'].apply(correct_date)

In [14]:
form_entry['created_at'] = form_entry['created_at'].apply(correct_date)

In [15]:
cohort_users_auth['datetime_cohort_user_creation'] = pd.to_datetime(cohort_users_auth['created_at'])

In [16]:
form_entry['datetime_form_entry_creation'] = pd.to_datetime(form_entry['created_at'])

In [17]:
cohort_users_auth

,email,role,created_at,cohort_id,datetime_cohort_user_creation
0,a+fakestudent8@4geeks.us,STUDENT,2020-08-19 23:06:31.519727,9,2020-08-19 23:06:31.519727
1,a+fakestudent8@4geeks.us,STUDENT,2020-08-19 23:06:31.532619,31,2020-08-19 23:06:31.532619
2,a+fakestudent8@4geeks.us,STUDENT,2020-08-19 23:06:31.538438,4,2020-08-19 23:06:31.538438
3,alejandro.leon1223@gmail.com,STUDENT,2020-08-19 23:06:31.550397,9,2020-08-19 23:06:31.550397
4,edgarbrignoni@icloud.com,STUDENT,2020-08-19 23:06:31.565350,9,2020-08-19 23:06:31.565350
...,...,...,...,...,...
7729,dierofortune2003@gmail.com,STUDENT,2023-01-31 18:21:04.258037,397,2023-01-31 18:21:04.258037
7730,yeshwanthk8826@gmail.com,STUDENT,2023-01-31 19:14:17.126300,61,2023-01-31 19:14:17.126300
7731,pablo@4geeks.co,STUDENT,2023-01-31 19:08:18.651832,523,2023-01-31 19:08:18.651832
7732,acelven@hotmail.com,STUDENT,2023-01-31 21:23:50.539551,61,2023-01-31 21:23:50.539551


In [18]:
merged = pd.merge(form_entry, cohort_users_auth, on='email', how='inner')

In [19]:
to_drop = merged[merged['datetime_cohort_user_creation'] > merged['datetime_form_entry_creation']]

In [20]:
form_entry.columns

Index(['id', 'first_name', 'last_name', 'email', 'phone', 'course',
       'client_comments', 'language', 'utm_url', 'utm_medium', 'utm_campaign',
       'street_address', 'country', 'city', 'latitude', 'longitude', 'state',
       'zip_code', 'storage_status', 'created_at', 'updated_at', 'contact_id',
       'lead_type', 'location', 'referral_key', 'automations', 'tags', 'gclid',
       'browser_lang', 'fb_ad_id', 'fb_adgroup_id', 'fb_form_id',
       'fb_leadgen_id', 'fb_page_id', 'utm_source', 'academy_id',
       'deal_status', 'sentiment', 'ac_contact_id', 'ac_deal_id', 'user_id',
       'won_at', 'ac_expected_cohort', 'current_download',
       'lead_generation_app_id', 'utm_content', 'storage_status_text',
       'custom_fields', 'sex', 'utm_placement', 'utm_plan', 'utm_term',
       'ac_deal_owner_full_name', 'ac_deal_owner_id',
       'ac_expected_cohort_date', 'ac_deal_amount', 'ac_deal_currency_code',
       'datetime_form_entry_creation'],
      dtype='object')

In [21]:
to_drop.columns

Index(['id', 'first_name', 'last_name', 'email', 'phone', 'course',
       'client_comments', 'language', 'utm_url', 'utm_medium', 'utm_campaign',
       'street_address', 'country', 'city', 'latitude', 'longitude', 'state',
       'zip_code', 'storage_status', 'created_at_x', 'updated_at',
       'contact_id', 'lead_type', 'location', 'referral_key', 'automations',
       'tags', 'gclid', 'browser_lang', 'fb_ad_id', 'fb_adgroup_id',
       'fb_form_id', 'fb_leadgen_id', 'fb_page_id', 'utm_source', 'academy_id',
       'deal_status', 'sentiment', 'ac_contact_id', 'ac_deal_id', 'user_id',
       'won_at', 'ac_expected_cohort', 'current_download',
       'lead_generation_app_id', 'utm_content', 'storage_status_text',
       'custom_fields', 'sex', 'utm_placement', 'utm_plan', 'utm_term',
       'ac_deal_owner_full_name', 'ac_deal_owner_id',
       'ac_expected_cohort_date', 'ac_deal_amount', 'ac_deal_currency_code',
       'datetime_form_entry_creation', 'role', 'created_at_y', 'cohort_i

In [22]:
to_drop = to_drop.drop(columns=['role', 'created_at_y', 'cohort_id', 'datetime_cohort_user_creation'])

In [23]:
to_drop = to_drop.rename(columns={'created_at_x':'created_at'})

In [24]:
merged = pd.merge(form_entry, to_drop, how='outer', indicator=True)
dropped = merged[merged['_merge'] == 'both'].drop('_merge', axis=1)
form_entry = merged[merged['_merge'] == 'left_only'].drop('_merge', axis=1)

In [25]:
form_entry = form_entry.drop_duplicates(subset=['email', 'course', 'language'], keep='last')

In [26]:
form_entry

,id,first_name,last_name,email,phone,course,client_comments,language,utm_url,utm_medium,...,sex,utm_placement,utm_plan,utm_term,ac_deal_owner_full_name,ac_deal_owner_id,ac_expected_cohort_date,ac_deal_amount,ac_deal_currency_code,datetime_form_entry_creation
2,135,Alejandro,Sanchez,a.a.lejo@gmail.com,NaN,full-stack,NaN,us,https://4geeksacademy.co/us/course/full-stack-...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-23 21:22:11.861392
4,171,José,Martínez,josemfcheo@gmail.com,NaN,full-stack,NaN,es,https://4geeksacademy.co/us/course/full-stack-...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-25 02:34:58.527898
10,33931,Carlos Coral Brito,NaN,coral.johnny.2@gmail.com,5.930994e+12,full-stack,NaN,en,https://4geeksacademy.com/es/coding-bootcamps/...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-12 05:00:54.847029
11,28178,christian muñoz,NaN,mazda3mps265@hotmail.com,6.871348e+08,full-stack,NaN,es,facebook_form_182669996627327,ppc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-03-20 10:45:04.424547
13,145,Marcelo,Ricigliano,marcelo.ricigliano@gmail.com,7.866130e+09,NaN,NaN,es,https://4geeksacademy.co/us/course/software-en...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-24 02:57:59.028751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71707,70124,Diego Cuñez Andrade,NaN,diegospion@hotmail.com,9.877922e+08,full-stack,NaN,es,facebook_form_1204581053520098,ppc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-17 12:34:02.041994
71708,70127,Ricardo Fernández,NaN,ricardto12@gmail.com,5.841268e+11,full-stack,NaN,en,https://4geeksacademy.com/es/aplica,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-17 13:15:21.768787
71709,70107,Renata A. de Oliveira,NaN,renatadeoliveiras.cursos@gmail.com,5.561995e+12,datascience-ml,NaN,en,https://4geeksacademy.com/es/coding-bootcamps/...,cpc,...,NaN,NaN,NaN,machine learning curso,NaN,NaN,NaN,NaN,NaN,2023-02-17 09:44:14.418626
71710,70128,NaN,NaN,samir.bo3ps3@gmail.com,NaN,NaN,NaN,en,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-17 13:39:12.605789


In [27]:
dropped

,id,first_name,last_name,email,phone,course,client_comments,language,utm_url,utm_medium,...,sex,utm_placement,utm_plan,utm_term,ac_deal_owner_full_name,ac_deal_owner_id,ac_expected_cohort_date,ac_deal_amount,ac_deal_currency_code,datetime_form_entry_creation
0,1423,Dylan,Stallone,stallonedylan954@gmail.com,9.544660e+09,NaN,NaN,us,https://4geeksacademy.co/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-28 07:46:58.469198
5,94,Alejandro,Sanchez,a.alejo@gmail.com,7.867313e+09,NaN,NaN,es,http://localhost:8000/us/apply,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-22 04:09:04.708149
6,94,Alejandro,Sanchez,a.alejo@gmail.com,7.867313e+09,NaN,NaN,es,http://localhost:8000/us/apply,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-22 04:09:04.708149
7,94,Alejandro,Sanchez,a.alejo@gmail.com,7.867313e+09,NaN,NaN,es,http://localhost:8000/us/apply,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-22 04:09:04.708149
8,94,Alejandro,Sanchez,a.alejo@gmail.com,7.867313e+09,NaN,NaN,es,http://localhost:8000/us/apply,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-09-22 04:09:04.708149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71483,64414,Héctor Silva,NaN,titocontactos@gmail.com,5.694511e+10,full-stack,NaN,en,https://4geeksacademy.com/es/coding-bootcamps/...,cpc,...,NaN,NaN,NaN,curso bootcamp,Dianela Barboza,49.0,NaN,NaN,NaN,2023-01-10 17:57:31.838442
71531,56419,Francisca Morales,NaN,fraanmoore@gmail.com,5.697352e+10,full-stack,NaN,en,https://4geeksacademy.com/us/coding-bootcamps/...,NaN,...,NaN,NaN,NaN,NaN,Jennifer Toledo,15.0,NaN,NaN,NaN,2022-11-07 20:09:01.775201
71532,56419,Francisca Morales,NaN,fraanmoore@gmail.com,5.697352e+10,full-stack,NaN,en,https://4geeksacademy.com/us/coding-bootcamps/...,NaN,...,NaN,NaN,NaN,NaN,Jennifer Toledo,15.0,NaN,NaN,NaN,2022-11-07 20:09:01.775201
71682,35128,Helena,NaN,helena.gonza@hotmail.com,3.462797e+10,full-stack,NaN,es,facebook_form_182669996627327,ppc,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-23 13:32:57.361720


#### **2. Clean some form_entry columns**

In [28]:
 # form_entry = pd.read_csv('../sources/All_Form_Entries - Sheet1.csv', low_memory=False)

In [29]:
form_entry['course'] = form_entry['course'].replace(['full-stack-ft', 'full_stack', 'full-stack', 'software-engineering',
                                         'coding-introduction', 'outcomes', 'Full Stack'], 'full-stack')

In [30]:
form_entry['course'] = form_entry['course'].replace(['machine-learning', 'machine-learning-enginnering', 'datascience-ml', 'datascience'], 
                                        'machine-learning-engineering')

In [31]:
form_entry['academy_id'] = np.where((form_entry['location'] == 'maracaibo-venezuela'),
                                2.0, form_entry['academy_id'])
form_entry['academy_id'] = np.where((form_entry['location'] == 'toronto-canada'),
                                4.0, form_entry['academy_id'])
form_entry['academy_id'] = np.where((form_entry['location'] == 'costa-rica') |
                                (form_entry['location'] == 'sanjose-uruguay'),
                                7.0, form_entry['academy_id'])

In [32]:
form_entry['fullname'] = form_entry['first_name'].fillna('') + str(' ') + form_entry['last_name'].fillna('')

In [33]:
form_entry['location'] = form_entry['location'].replace(['maracaibo'], 'maracaibo-venezuela')

In [34]:
form_entry['location'] = form_entry['location'].replace(['los-cortijos-caracas'], 'caracas-venezuela')

In [35]:
form_entry['location'] = form_entry['location'].replace(['lisboa-portugal'], 'lisbon-portugal')

In [36]:
form_entry['language'] = form_entry['language'].replace('us', 'en')

In [37]:
# Delete "+00:00"
def correct_date(time):
    if pd.isna(time):
         return time
    else:
        sep = '+'
        stripped = time.split(sep, 1)[0]
        return stripped

In [38]:
form_entry['created_at'] = form_entry['created_at'].apply(correct_date)

In [39]:
form_entry['updated_at'] = form_entry['updated_at'].apply(correct_date)

In [40]:
form_entry['ac_expected_cohort_date'] = form_entry['ac_expected_cohort_date'].apply(correct_date)

#### **3. Identify upcoming cohorts and select only needed columns**

In [41]:
cohorts = pd.read_csv('../sources/cohorts.csv')

In [42]:
cohorts.columns

Index(['id', 'slug', 'name', 'kickoff_date', 'ending_date', 'current_day',
       'stage', 'language', 'created_at', 'updated_at', 'academy_id',
       'timezone', 'private', 'never_ends', 'schedule_id',
       'syllabus_version_id', 'online_meeting_url', 'remote_available',
       'current_module', 'history_log', 'is_hidden_on_prework'],
      dtype='object')

In [43]:
cohorts['academy_id'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 389 entries, 0 to 388
Series name: academy_id
Non-Null Count  Dtype
--------------  -----
389 non-null    int64
dtypes: int64(1)
memory usage: 3.2 KB


In [44]:
# Delete "+00:00"
def correct_date(time):
    if pd.isna(time):
         return time
    else:
        sep = '+'
        stripped = time.split(sep, 1)[0]
        return stripped

In [45]:
cohorts['kickoff_date'] = cohorts['kickoff_date'].apply(correct_date)

In [46]:
cohorts['datetime_kickoff'] = pd.to_datetime(cohorts['kickoff_date'])

In [47]:
current_time = pd.Timestamp.now()

In [48]:
upcoming_cohorts = cohorts[cohorts['datetime_kickoff']>current_time]

In [ ]:
upcoming_cohorts = upcoming_cohorts[upcoming_cohorts['never_ends']==False]

In [49]:
upcoming_cohorts = upcoming_cohorts[(upcoming_cohorts['stage']!='DELETED')]

In [50]:
upcoming_cohorts = upcoming_cohorts[['slug', 'academy_id', 'datetime_kickoff']]

In [51]:
upcoming_cohorts

,slug,academy_id,datetime_kickoff
124,spain-fs-pt-enero,6,2023-06-06 14:16:00
191,latam-pt-ml-2,7,2023-03-20 19:55:00
215,spain-fs-pt-marzo,6,2023-03-07 15:23:00
221,latam-pt-20,7,2023-06-19 18:16:00
222,latam-pt-17,7,2023-03-13 18:12:00
262,spain-fs-pt-abril,6,2023-04-10 15:24:00
266,miami-50,4,2023-03-20 14:55:00
267,latam-pt-18,7,2023-04-17 18:14:00
293,spain-fs-pt-mayo,6,2023-05-02 15:26:00
308,latam-pt-26,7,2023-12-11 18:36:00


#### **4. Complete ac_expected_cohort and add cohort_assignation_error**

In [52]:
form_entry['datetime_expected_cohort_date'] = pd.to_datetime(form_entry['ac_expected_cohort_date'])

In [53]:
def assign_expected_cohort(row, df=upcoming_cohorts):
    
    datetime_expected = row['datetime_expected_cohort_date']
    expected_date = row['ac_expected_cohort_date']
    ac_expected_cohort = row['ac_expected_cohort']
    fe_academy_id = row['academy_id']
    fe_course = row['course']

    if pd.notnull(ac_expected_cohort):
        return ac_expected_cohort
    elif pd.isna(expected_date):
        return np.nan
    elif pd.isna(fe_academy_id):
        return 'Missing academy'
    elif pd.isna(fe_course):
        return 'Missing course'
    else:

        df_filtered = df[df['academy_id']==int(fe_academy_id)]

        if fe_course == 'machine-learning-engineering':
            df_filtered = df_filtered[df_filtered['slug'].str.contains('-ml-')]
        else:
            df_filtered = df_filtered[~df_filtered['slug'].str.contains('-ml-')]

        if df_filtered.empty:
            return 'Missing upcoming cohort'
        else:
            datetime_expected = datetime_expected.replace(tzinfo=None)
            df_filtered = df_filtered[df_filtered['datetime_kickoff'] >= datetime_expected]
            df_filtered['time_diff'] = abs(df_filtered['datetime_kickoff'] - datetime_expected)    
            min_idx = df_filtered['time_diff'].idxmin()
            expected_cohort = df_filtered.loc[min_idx, 'slug']
            return expected_cohort

In [54]:
form_entry['date_assigned_cohort'] = form_entry.apply(assign_expected_cohort, axis=1)

In [55]:
error_values = ['Missing academy', 'Missing course', 'Missing upcoming cohort']

In [56]:
form_entry.loc[form_entry['date_assigned_cohort'].isin(error_values), 'cohort_assignation_error'] = form_entry['date_assigned_cohort']

In [57]:
form_entry.loc[~form_entry['date_assigned_cohort'].isin(error_values), 'ac_expected_cohort'] = form_entry['date_assigned_cohort']

In [58]:
form_entry.loc[form_entry['date_assigned_cohort'].isna(), ['ac_expected_cohort', 'cohort_assignation_error']] = np.NaN

In [59]:
form_entry = form_entry.drop(columns=['date_assigned_cohort'])

#### **5. Look for inconsistencies between academy and location**

In [60]:
academies_alias = pd.read_csv('../sources/academies_alias.csv')

In [61]:
dict_locations = academies_alias.groupby('name')['slug'].apply(list).to_dict()

In [62]:
academies_alias = academies_alias.drop(columns=['slug', 'status'])

In [63]:
academies_alias = academies_alias.rename(columns={'name':'campus'})

In [64]:
academies_alias = academies_alias.drop_duplicates()

In [65]:
academies_alias

,academy_id,campus
0,2,4Geeks Academy Caracas
2,4,4Geeks Academy Miami
4,11,4Geeks Academy Orlando
5,10,4Geeks Academy Europe
13,6,4Geeks Academy Madrid
17,5,4Geeks Academy Santiago
18,7,4Geeks Academy Latam


In [66]:
form_entry = form_entry.merge(academies_alias, on='academy_id', how='left')

In [67]:
def campus_check(row):
    
    campus = row['campus']
    location = row['location']
    
    if pd.isna(campus):
        return 'No campus assigned'
    else:
        if campus in dict_locations:
            if location in dict_locations[campus]:
                return 'Location matches campus'
            else:
                return 'Location does not match campus'
        else:
            return 'Unknown campus'

In [68]:
form_entry['campus_check'] = form_entry.apply(campus_check, axis=1)

In [69]:
form_entry.columns

Index(['id', 'first_name', 'last_name', 'email', 'phone', 'course',
       'client_comments', 'language', 'utm_url', 'utm_medium', 'utm_campaign',
       'street_address', 'country', 'city', 'latitude', 'longitude', 'state',
       'zip_code', 'storage_status', 'created_at', 'updated_at', 'contact_id',
       'lead_type', 'location', 'referral_key', 'automations', 'tags', 'gclid',
       'browser_lang', 'fb_ad_id', 'fb_adgroup_id', 'fb_form_id',
       'fb_leadgen_id', 'fb_page_id', 'utm_source', 'academy_id',
       'deal_status', 'sentiment', 'ac_contact_id', 'ac_deal_id', 'user_id',
       'won_at', 'ac_expected_cohort', 'current_download',
       'lead_generation_app_id', 'utm_content', 'storage_status_text',
       'custom_fields', 'sex', 'utm_placement', 'utm_plan', 'utm_term',
       'ac_deal_owner_full_name', 'ac_deal_owner_id',
       'ac_expected_cohort_date', 'ac_deal_amount', 'ac_deal_currency_code',
       'datetime_form_entry_creation', 'fullname',
       'datetime_expect

In [70]:
form_entry.to_csv('../output/form_entry_processed.csv', index=False)